<a href="https://colab.research.google.com/github/OrangeBaron/Manuscript-Restorer/blob/main/Manuscript_Restorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 1. Installazione e Setup
# @markdown Installa le librerie necessarie e importa i moduli.

!apt-get install -y poppler-utils
!pip install -q opencv-python-headless matplotlib pdf2image

import cv2
import numpy as np
import os
import shutil
import glob
from google.colab import files
from matplotlib import pyplot as plt
from pdf2image import convert_from_path
from IPython.display import HTML, display
import base64
from io import BytesIO
from PIL import Image as PILImage

print("✅ Dipendenze installate e moduli importati.")

In [ ]:
# @title 2. Caricamento Documenti
# @markdown Carica file JPG, PNG o PDF.

INPUT_FOLDER = "input_docs"
OUTPUT_FOLDER = "restored_docs"
PDF_DPI = 300 # @param {type:"slider", min:150, max:600, step:50}
# @markdown *DPI Conversione PDF: 300 è standard, 400-500 consigliato per manoscritti molto piccoli.*

# 1. Pulizia ambiente
if os.path.exists(INPUT_FOLDER): shutil.rmtree(INPUT_FOLDER)
if os.path.exists(OUTPUT_FOLDER): shutil.rmtree(OUTPUT_FOLDER)
os.makedirs(INPUT_FOLDER)
os.makedirs(OUTPUT_FOLDER)

# 2. Upload
print(f"📂 Seleziona le immagini o i PDF da caricare...")
uploaded = files.upload()

if not uploaded:
    print("⚠️ Nessun file caricato.")
else:
    print(f"\n🔄 Analisi file caricati (DPI target: {PDF_DPI})...")

    for filename, data in uploaded.items():
        temp_path = os.path.join(INPUT_FOLDER, filename)
        with open(temp_path, 'wb') as f:
            f.write(data)

        # Gestione PDF
        if filename.lower().endswith(".pdf"):
            print(f"   📄 Elaborazione PDF: {filename}...")
            try:
                safe_base_name = os.path.splitext(filename)[0]

                # Conversione in PNG (Cruciale per il restauro: evita artefatti JPG)
                convert_from_path(
                    temp_path,
                    dpi=PDF_DPI,
                    output_folder=INPUT_FOLDER,
                    fmt='png',
                    output_file=safe_base_name,
                    paths_only=True
                )

                print(f"      -> Pagine estratte come PNG (Massima qualità).")
                os.remove(temp_path)

            except Exception as e:
                print(f"   ❌ Errore conversione PDF {filename}: {e}")
        else:
            print(f"   🖼️ Immagine pronta: {filename}")

    # Conteggio finale
    count = len(glob.glob(os.path.join(INPUT_FOLDER, "*")))
    print(f"\n✅ Tutto pronto! {count} pagine pronte in '{INPUT_FOLDER}'.")

In [ ]:
# @title 3. Test Parametri e Anteprima
# @markdown Regola i parametri qui sotto e premi "Play" per vedere l'effetto su un campione.

# --- PARAMETRI DI RESTAURO SPECIFICI ---

# @markdown ### 1. RECUPERO STRUTTURA (Inchiostro)
INK_THICKNESS = 1 # @param {type:"slider", min:0, max:3, step:1}
# @markdown **Cosa fa:** Aggiunge corpo ai tratti esistenti.
# @markdown * **0:** Nessuna modifica.
# @markdown * **1:** Consigliato per penna fine o sbiadita.
# @markdown * **2-3:** Solo per testo spezzettato (può unire le lettere piccole).

ADAPTIVE_THRESHOLDING = True # @param {type:"boolean"}
ADAPTIVE_SENSITIVITY = 7 # @param {type:"slider", min:2, max:20, step:1}
# @markdown **Cosa fa:** Regola quanto il sistema è "sensibile" nel riconoscere l'inchiostro rispetto allo sfondo locale.
# @markdown * **Valori BASSI (3-7):** Fondamentale per **testo molto sbiadito**. Recupera tratti quasi invisibili, ma potrebbe scambiare macchie della carta per testo.
# @markdown * **Valori STANDARD (8-12):** Buon bilanciamento.
# @markdown * **Valori ALTI (15+):** Pulizia aggressiva. Ottimo se il foglio è molto sporco ma l'inchiostro è ben marcato. Rischia di cancellare scritte leggere.

# @markdown ---
# @markdown ### 2. PULIZIA SFONDO (Carta)
DENOISE_STRENGTH = 2 # @param {type:"slider", min:0, max:20, step:1}
# @markdown **Cosa fa:** Rimuove la granulosità fine. Tenere basso (0-3) per documenti storici per non perdere dettagli.

HIGH_PASS_KERNEL = 101 # @param {type:"slider", min:11, max:201, step:10}
# @markdown **Cosa fa:** Appiattisce l'illuminazione (toglie ombre e ingiallimento).
# @markdown * **100+:** Pulizia delicata, mantiene la "texture" del testo.
# @markdown * **<50:** Molto aggressivo, rende lo sfondo bianco puro ma può "svuotare" le lettere larghe.

# @markdown ---
# @markdown ### 3. FINITURA (Contrasto e Nitidezza)
GAMMA = 0.6 # @param {type:"slider", min:0.1, max:2.0, step:0.1}
# @markdown **Cosa fa:** Scurisce i mezzi toni. < 1.0 scurisce il grigio rendendolo nero.

SHARPEN_AMOUNT = 0.5 # @param {type:"slider", min:0.0, max:3.0, step:0.1}
# @markdown **Cosa fa:** Rende i bordi delle lettere più croccanti.

# @markdown ---
# @markdown ### OPZIONI VISUALIZZAZIONE
PREVIEW_CROP_SIZE = 1200 # @param {type:"integer"}
PREVIEW_OFFSET_X = 0 # @param {type:"slider", min:-1, max:1, step:0.1}
PREVIEW_OFFSET_Y = 0 # @param {type:"slider", min:-1, max:1, step:0.1}
OUTPUT_NEGATIVE = False # @param {type:"boolean"}


def get_manual_crop(image, crop_size, offset_x, offset_y):
    h, w = image.shape[:2]
    if h <= crop_size or w <= crop_size: return image

    # Calcolo dell'offset massimo in pixel (dal centro)
    # L'area spostabile è la differenza tra dimensione immagine e crop, diviso 2
    max_shift_x = (w - crop_size) // 2
    max_shift_y = (h - crop_size) // 2

    # Conversione da valore slider (-1.0 a 1.0) a pixel effettivi
    pixel_offset_x = int(offset_x * max_shift_x)
    pixel_offset_y = int(offset_y * max_shift_y)

    # Calcolo coordinate centro basate su offset calcolato
    center_x = (w // 2) + pixel_offset_x
    center_y = (h // 2) + pixel_offset_y

    # Calcolo angolo top-left
    start_x = center_x - (crop_size // 2)
    start_y = center_y - (crop_size // 2)

    # Vincoli per non uscire dall'immagine (Clamping)
    start_x = max(0, min(start_x, w - crop_size))
    start_y = max(0, min(start_y, h - crop_size))

    return image[start_y:start_y+crop_size, start_x:start_x+crop_size]


def apply_restoration(img, denoise_h, high_pass_k, ink_thick, use_adaptive, adaptive_sens, gamma, sharpen_amt):

    # 1. CONVERSIONE CANALI (Logica Ferrogallica)
    if len(img.shape) == 3:
        b, g, r = cv2.split(img)
        # L'inchiostro ferrogallico è scuro nel blu e verde, il rosso contiene il rumore della carta
        gray = cv2.addWeighted(g, 0.5, b, 0.5, 0)
    else:
        gray = img.copy()

    # 2. DILATAZIONE
    if ink_thick > 0:
        # Erode scurisce (perché l'inchiostro è nero su bianco)
        kernel = np.ones((2,2), np.uint8)
        gray = cv2.erode(gray, kernel, iterations=ink_thick)

    # 3. DENOISE LEGGERO
    if denoise_h > 0:
        gray = cv2.fastNlMeansDenoising(gray, None, h=denoise_h, templateWindowSize=7, searchWindowSize=21)

    # 4. HIGH PASS FILTER (Division Normalization)
    # Calcoliamo lo sfondo sfocato e dividiamo. Questo rimuove le ombre mantenendo il testo.
    if high_pass_k % 2 == 0: high_pass_k += 1 # Assicura dispari

    # Usiamo float32 per precisione matematica
    gray_float = gray.astype(np.float32)
    bg_blur = cv2.GaussianBlur(gray_float, (high_pass_k, high_pass_k), 0)

    # Division Normalization con epsilon per evitare divisioni per zero
    processed = 255 * (gray_float / (bg_blur + 1e-5))
    processed = np.clip(processed, 0, 255).astype(np.uint8)

    # 5. ADAPTIVE THRESHOLD BLENDING
    if use_adaptive:
        # Creiamo una maschera binaria locale
        block_size = 25 # Dimensione finestra locale

        # Qui usiamo il parametro ADAPTIVE_SENSITIVITY (C constant) invece di un 10 fisso
        local_thresh = cv2.adaptiveThreshold(
            processed, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY, block_size, adaptive_sens
        )

        # Invece di sottrarre, usiamo Multiply per "bruciare" l'inchiostro sulla carta
        mask_inv = cv2.bitwise_not(local_thresh) # Testo è bianco qui
        # Sfocatura maschera per blend morbido
        mask_blur = cv2.GaussianBlur(mask_inv, (3,3), 0)

        # Sovrapposizione: Dove la maschera è bianca (testo), scurisci l'immagine base
        processed = processed.astype(float)
        darkener = mask_blur.astype(float) / 255.0
        # Formula: Pixel = Pixel - (Pixel * Darkener * Intensità)
        processed = processed - (processed * darkener * 0.8)
        processed = np.clip(processed, 0, 255).astype(np.uint8)

    # 6. GAMMA CORRECTION
    if gamma != 1.0:
        invGamma = 1.0 / gamma
        table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
        processed = cv2.LUT(processed, table)

    # 7. SHARPENING FINALE
    if sharpen_amt > 0:
        gaussian = cv2.GaussianBlur(processed, (0, 0), 3.0)
        processed = cv2.addWeighted(processed, 1.0 + sharpen_amt, gaussian, -sharpen_amt, 0)

    return cv2.cvtColor(processed, cv2.COLOR_GRAY2BGR)


# --- FUNZIONI PER ANTEPRIMA INTERATTIVA ---

def cv2_to_base64(img_cv2):
    img_rgb = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)
    pil_img = PILImage.fromarray(img_rgb)
    buffer = BytesIO()
    pil_img.save(buffer, format="JPEG", quality=95)
    img_str = base64.b64encode(buffer.getvalue()).decode("utf-8")
    return f"data:image/jpeg;base64,{img_str}"

def show_comparison_slider(img_before, img_after):
    h, w = img_before.shape[:2]

    # Prepara le immagini base64
    src_before = cv2_to_base64(img_before)
    src_after = cv2_to_base64(img_after)

    uid = np.random.randint(100000)

    html_code = f"""
    <div style="position: relative; width: {w}px; height: {h}px; margin: 0 auto; border: 2px solid #333;">

        <img src="{src_after}" style="position: absolute; top: 0; left: 0; width: {w}px; height: {h}px;">

        <div id="img-wrapper-{uid}" style="position: absolute; top: 0; left: 0; height: {h}px; width: 50%; overflow: hidden; border-right: 2px solid rgba(255,255,255,0.8); box-sizing: border-box;">
            <img src="{src_before}" style="width: {w}px; height: {h}px; max-width: none;">
        </div>

        <input type="range" min="0" max="{w}" value="{w//2}" id="slider-{uid}"
               style="position: absolute; top: 0; left: 0; width: 100%; height: 100%; opacity: 0; cursor: col-resize; margin: 0;">
    </div>

    <script>
        document.getElementById('slider-{uid}').oninput = function() {{
            document.getElementById('img-wrapper-{uid}').style.width = this.value + 'px';
        }};
    </script>
    """
    display(HTML(html_code))


# --- ESECUZIONE ANTEPRIMA ---
list_files = sorted(glob.glob(os.path.join(INPUT_FOLDER, "*")))
if not list_files:
    print("❌ Nessun file trovato. Torna allo Step 2.")
else:
    sample_file = list_files[0]
    img_original = cv2.imread(sample_file, 1)
    if img_original is None:
        print(f"❌ Errore lettura: {sample_file}")
    else:
        print(f"🔍 Generazione anteprima...")

        # Crop Manuale
        img_crop = get_manual_crop(img_original, PREVIEW_CROP_SIZE, PREVIEW_OFFSET_X, PREVIEW_OFFSET_Y)

        # Restauro
        img_restored_crop = apply_restoration(
            img_crop.copy(),
            DENOISE_STRENGTH, HIGH_PASS_KERNEL, INK_THICKNESS,
            ADAPTIVE_THRESHOLDING, ADAPTIVE_SENSITIVITY,
            GAMMA, SHARPEN_AMOUNT
        )

        if OUTPUT_NEGATIVE:
            img_restored_crop = cv2.bitwise_not(img_restored_crop)

        # Mostriamo lo slider
        show_comparison_slider(img_crop, img_restored_crop)

In [ ]:
# @title 4. Elaborazione Batch e Download
# @markdown Esegue il restauro su tutti i file e crea uno ZIP.

# --- CONTROLLI DI SICUREZZA ---
# Impostiamo valori di default se le celle precedenti non sono state eseguite
if 'OUTPUT_NEGATIVE' not in globals(): OUTPUT_NEGATIVE = False
if 'GAMMA' not in globals(): GAMMA = 0.6
if 'SHARPEN_AMOUNT' not in globals(): SHARPEN_AMOUNT = 0.5
if 'DENOISE_STRENGTH' not in globals(): DENOISE_STRENGTH = 2
if 'INK_THICKNESS' not in globals(): INK_THICKNESS = 1
if 'HIGH_PASS_KERNEL' not in globals(): HIGH_PASS_KERNEL = 101
if 'ADAPTIVE_THRESHOLDING' not in globals(): ADAPTIVE_THRESHOLDING = True
if 'ADAPTIVE_SENSITIVITY' not in globals(): ADAPTIVE_SENSITIVITY = 7

# --- FEEDBACK UTENTE ---
mode_str = "NEGATIVO (Sfondo Nero)" if OUTPUT_NEGATIVE else "POSITIVO (Sfondo Bianco)"
print(f"🚀 Inizio elaborazione batch...")
print(f"⚙️ Output: {mode_str}")
print(f"⚙️ Strategia: Ink={INK_THICKNESS}, AdaptSens={ADAPTIVE_SENSITIVITY}, HighPass={HIGH_PASS_KERNEL}, Gamma={GAMMA}")

processed_count = 0
file_list = sorted(glob.glob(os.path.join(INPUT_FOLDER, "*")))

# Pulizia cartella output precedente
if os.path.exists(OUTPUT_FOLDER):
    shutil.rmtree(OUTPUT_FOLDER)
os.makedirs(OUTPUT_FOLDER)

if not file_list:
    print("❌ Nessun file trovato nella cartella di input.")
else:
    for file_path in file_list:
        filename = os.path.basename(file_path)
        img = cv2.imread(file_path)

        if img is None:
            print(f"⚠️ Errore lettura: {filename}")
            continue

        try:
            # 1. Applicazione Restauro
            img_clean = apply_restoration(
                img,
                DENOISE_STRENGTH,
                HIGH_PASS_KERNEL,
                INK_THICKNESS,
                ADAPTIVE_THRESHOLDING,
                ADAPTIVE_SENSITIVITY,
                GAMMA,
                SHARPEN_AMOUNT
            )

            # 2. Inversione se richiesto
            if OUTPUT_NEGATIVE:
                img_clean = cv2.bitwise_not(img_clean)

            # 3. Generazione Nome File Intelligente
            suffix = ""
            if INK_THICKNESS > 0: suffix += "_thick"
            if OUTPUT_NEGATIVE: suffix += "_neg"

            # Mantiene l'estensione originale o forza jpg
            name_base = os.path.splitext(filename)[0]
            save_name = f"{name_base}_restored{suffix}.jpg"
            save_path = os.path.join(OUTPUT_FOLDER, save_name)

            # Salvataggio alta qualità (JPG q=95)
            cv2.imwrite(save_path, img_clean, [int(cv2.IMWRITE_JPEG_QUALITY), 95])

            processed_count += 1
            print(f" -> ✅ Fatto: {filename}")

        except NameError:
             print("❌ ERRORE CRITICO: Devi eseguire prima la Cella 3 per caricare la funzione 'apply_restoration' aggiornata!")
             break
        except TypeError as e:
             print(f"❌ ERRORE CRITICO: I parametri della funzione non corrispondono. Dettaglio: {e}")
             print("💡 SUGGERIMENTO: Hai aggiornato la Cella 3? Assicurati di averla eseguita dopo le modifiche.")
             break
        except Exception as e:
            print(f"❌ Errore generico su {filename}: {e}")

    # Creazione ZIP e Download
    if processed_count > 0:
        print(f"\n📦 Creazione archivio ZIP ({processed_count} file)...")
        zip_filename = "documenti_restaurati"

        if os.path.exists(f"{zip_filename}.zip"):
            os.remove(f"{zip_filename}.zip")

        shutil.make_archive(zip_filename, 'zip', OUTPUT_FOLDER)
        print("⬇️ Download automatico in corso...")
        files.download(f"{zip_filename}.zip")
    else:
        print("❌ Nessun file prodotto.")